In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.regularizers import l2

df = pd.read_csv('../dataset/cleaned.csv')

In [4]:
# getting dummie variables

df_dummies = df.filter(['city', 'state', 'event_type', 'flight_condition','flight_operation', 'sightseeing', 'second_pilot'], axis=1)
df_dummies = pd.get_dummies(df_dummies, columns=['city', 'state', 'event_type', 'flight_condition','flight_operation', 'sightseeing', 'second_pilot'], drop_first=True)

In [5]:
# set any deaths or injuries to 1

for i in range(len(df['deaths_and_injuries'])):
    if df['deaths_and_injuries'][i] > 0:
        df['deaths_and_injuries'][i] = 1

<ipython-input-5-3ff8f6f3afb8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['deaths_and_injuries'][i] = 1


In [5]:
# setting up X and y

X = df_dummies.values
y = df['deaths_and_injuries'].astype(int).values

In [6]:
# train test split and scale
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [7]:
X_train[0].shape[0]

5166

===

### Model 1

In [7]:
def model_func(l1=32, l2=32, l3=32, l4=32, l5=32, drop=.5):
    
    print('layer_1', l1, 'layer_2', l2, 'layer_3', l3, 'layer_4', l4, 'layer_5', l5)
    
    model = Sequential()
    
    model.add(Dense(l1, activation='relu', input_shape=(5166,)))
    model.add(Dropout(drop))
    
    model.add(Dense(l2, activation='relu'))
    model.add(Dropout(drop))
 
    model.add(Dense(l3, activation='relu'))
    model.add(Dropout(drop))

    model.add(Dense(l4, activation='relu'))
    model.add(Dropout(drop))

    model.add(Dense(l5, activation='relu'))
    model.add(Dropout(drop))    

    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics='acc')
    
    return model

nn = KerasRegressor(build_fn=model_func, batch_size=256, verbose=0)

params = {
    'epochs': [50],
    'l1': [4096],
    'l2': [2048],
    'l3': [1024],
    'l4': [512],
    'l5': [256],
    'drop': [.75, .8, 85]
}

gs1 = GridSearchCV(nn, param_grid=params, cv=5)

gs1.fit(X_train, y_train)

print(gs.best_score_) # <- negative mean squared error, ie the higher (less negative) the better
gs1.best_params_

<ipython-input-7-3e053e861464>:28: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  nn = KerasRegressor(build_fn=model_func, batch_size=256, verbose=1)


layer_1 4096 layer_2 2048 layer_3 1024 layer_4 512 layer_5 256
Epoch 1/50
34/34 [==============================] - 14s 341ms/step - loss: 0.7134 - acc: 0.5239
Epoch 2/50
34/34 [==============================] - 12s 349ms/step - loss: 0.6998 - acc: 0.5292
Epoch 3/50
34/34 [==============================] - 12s 343ms/step - loss: 0.6928 - acc: 0.5294
Epoch 4/50
34/34 [==============================] - 12s 342ms/step - loss: 0.6906 - acc: 0.5375
Epoch 5/50
34/34 [==============================] - 11s 326ms/step - loss: 0.6873 - acc: 0.5414
Epoch 6/50
34/34 [==============================] - 11s 327ms/step - loss: 0.6837 - acc: 0.5489
Epoch 7/50
34/34 [==============================] - 11s 332ms/step - loss: 0.6755 - acc: 0.5503
Epoch 8/50
34/34 [==============================] - 12s 345ms/step - loss: 0.6676 - acc: 0.5612
Epoch 9/50
34/34 [==============================] - 11s 333ms/step - loss: 0.6456 - acc: 0.6120
Epoch 10/50
34/34 [==============================] - 11s 324ms/step - los

34/34 [==============================] - 12s 352ms/step - loss: 0.4468 - acc: 0.8072
Epoch 15/50
34/34 [==============================] - 11s 327ms/step - loss: 0.4220 - acc: 0.8171
Epoch 16/50
34/34 [==============================] - 11s 332ms/step - loss: 0.3919 - acc: 0.8250
Epoch 17/50
34/34 [==============================] - 11s 330ms/step - loss: 0.3704 - acc: 0.8345
Epoch 18/50
34/34 [==============================] - 11s 331ms/step - loss: 0.3475 - acc: 0.8420
Epoch 19/50
34/34 [==============================] - 11s 333ms/step - loss: 0.3242 - acc: 0.8531
Epoch 20/50
34/34 [==============================] - 11s 317ms/step - loss: 0.3118 - acc: 0.8502
Epoch 21/50
34/34 [==============================] - 8s 225ms/step - loss: 0.3068 - acc: 0.8552
Epoch 22/50
34/34 [==============================] - 10s 306ms/step - loss: 0.2964 - acc: 0.8563
Epoch 23/50
34/34 [==============================] - 11s 332ms/step - loss: 0.2909 - acc: 0.8613
Epoch 24/50
34/34 [========================

34/34 [==============================] - 7s 197ms/step - loss: 0.3942 - acc: 0.8332 4s - l
Epoch 29/50
34/34 [==============================] - 7s 199ms/step - loss: 0.3828 - acc: 0.8315 2s - loss: 0.3870 
Epoch 30/50
34/34 [==============================] - 7s 198ms/step - loss: 0.3713 - acc: 0.8386
Epoch 31/50
34/34 [==============================] - 7s 195ms/step - loss: 0.3616 - acc: 0.8423
Epoch 32/50
34/34 [==============================] - 7s 198ms/step - loss: 0.3516 - acc: 0.8507
Epoch 33/50
34/34 [==============================] - 7s 199ms/step - loss: 0.3352 - acc: 0.8579
Epoch 34/50
34/34 [==============================] - 7s 199ms/step - loss: 0.3284 - acc: 0.8579
Epoch 35/50
34/34 [==============================] - 7s 197ms/step - loss: 0.3308 - acc: 0.8538 4s - lo
Epoch 36/50
34/34 [==============================] - 7s 201ms/step - loss: 0.3195 - acc: 0.8600 2s - loss: 0.3298 
Epoch 37/50
34/34 [==============================] - 7s 196ms/step - loss: 0.3202 - acc: 0.8558

34/34 [==============================] - 7s 198ms/step - loss: 0.2807 - acc: 0.8765
Epoch 43/50
34/34 [==============================] - 7s 201ms/step - loss: 0.2777 - acc: 0.8727
Epoch 44/50
34/34 [==============================] - 7s 198ms/step - loss: 0.2783 - acc: 0.8681
Epoch 45/50
34/34 [==============================] - 7s 201ms/step - loss: 0.2660 - acc: 0.8699
Epoch 46/50
34/34 [==============================] - 7s 207ms/step - loss: 0.2696 - acc: 0.8704
Epoch 47/50
34/34 [==============================] - 7s 202ms/step - loss: 0.2635 - acc: 0.8734
Epoch 48/50
34/34 [==============================] - 7s 197ms/step - loss: 0.2659 - acc: 0.8754
Epoch 49/50
34/34 [==============================] - 7s 199ms/step - loss: 0.2651 - acc: 0.8701 4s - lo
Epoch 50/50
9/9 [==============================] - 1s 54ms/step - loss: 1.3877 - acc: 0.5223
layer_1 4096 layer_2 2048 layer_3 1024 layer_4 512 layer_5 256
layer_1 4096 layer_2 2048 layer_3 1024 layer_4 512 layer_5 256


...

C:\Users\Henry\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Henry\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Henry\anaconda3\lib\site-packages\keras\wrappers\scikit_learn.py", line 155, in fit
    self.model = self.build_fn(**self.filter_sk_params(self.build_fn))
  File "<ipython-input-7-3e053e861464>", line 8, in model_func
    model.add(Dropout(drop))
  File "C:\Users\Henry\anaconda3\lib\site-packages\keras\layers\core\dropout.py", line 80, in __init__
    raise ValueError(f'Invalid value {rate} received for '
ValueError: Invalid value 85 received for `rate`, expected a value between 0 and 1.

  warnings.warn("Estimator fit failed. The score on this train-test

layer_1 4096 layer_2 2048 layer_3 1024 layer_4 512 layer_5 256
layer_1 4096 layer_2 2048 layer_3 1024 layer_4 512 layer_5 256


C:\Users\Henry\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Henry\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Henry\anaconda3\lib\site-packages\keras\wrappers\scikit_learn.py", line 155, in fit
    self.model = self.build_fn(**self.filter_sk_params(self.build_fn))
  File "<ipython-input-7-3e053e861464>", line 8, in model_func
    model.add(Dropout(drop))
  File "C:\Users\Henry\anaconda3\lib\site-packages\keras\layers\core\dropout.py", line 80, in __init__
    raise ValueError(f'Invalid value {rate} received for '
ValueError: Invalid value 85 received for `rate`, expected a value between 0 and 1.

  warnings.warn("Estimator fit failed. The score on this train-test

Epoch 1/50
42/42 [==============================] - 9s 200ms/step - loss: 0.7342 - acc: 0.5161
Epoch 2/50
42/42 [==============================] - 9s 202ms/step - loss: 0.7025 - acc: 0.5170
Epoch 3/50
42/42 [==============================] - 8s 201ms/step - loss: 0.6955 - acc: 0.5268
Epoch 4/50
42/42 [==============================] - 9s 209ms/step - loss: 0.6930 - acc: 0.5328
Epoch 5/50
42/42 [==============================] - 9s 222ms/step - loss: 0.6912 - acc: 0.5342
Epoch 6/50
42/42 [==============================] - 8s 200ms/step - loss: 0.6902 - acc: 0.5296
Epoch 7/50
42/42 [==============================] - 9s 203ms/step - loss: 0.6862 - acc: 0.5406
Epoch 8/50
42/42 [==============================] - 8s 199ms/step - loss: 0.6837 - acc: 0.5438
Epoch 9/50
42/42 [==============================] - 9s 204ms/step - loss: 0.6810 - acc: 0.5432
Epoch 10/50
42/42 [==============================] - 9s 204ms/step - loss: 0.6760 - acc: 0.5601
Epoch 11/50
42/42 [==============================

{'drop': 0.8,
 'epochs': 50,
 'l1': 4096,
 'l2': 2048,
 'l3': 1024,
 'l4': 512,
 'l5': 256}

===

### Model 2

In [9]:
def model_func(l1=32, l2=32, l3=32, l4=32, l5=32, drop=.5):
    
    print('layer_1', l1, 'layer_2', l2, 'layer_3', l3, 'layer_4', l4, 'layer_5', l5)
    
    model = Sequential()
    
    model.add(Dense(l1, activation='relu', input_shape=(5166,)))
    model.add(Dropout(drop))
    
    model.add(Dense(l2, activation='relu'))
    model.add(Dropout(drop))
 
    model.add(Dense(l3, activation='relu'))
    model.add(Dropout(drop))

    model.add(Dense(l4, activation='relu'))
    model.add(Dropout(drop))

    model.add(Dense(l5, activation='relu'))
    model.add(Dropout(drop))    

    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics='acc')
    
    return model

nn = KerasRegressor(build_fn=model_func, batch_size=256, verbose=0)

params = {
    'epochs': [50],
    'l1': [4096],
    'l2': [1024],
    'l3': [256],
    'l4': [64],
    'l5': [16],
    'drop': [.75, .8, 85]
}

gs2 = GridSearchCV(nn, param_grid=params, cv=5)

gs2.fit(X_train, y_train)

print(gs2.best_score_) # <- negative mean squared error, ie the higher (less negative) the better
gs2.best_params_

layer_1 4096 layer_2 1024 layer_3 256 layer_4 64 layer_5 16


<ipython-input-9-beb3aba4a063>:28: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  nn = KerasRegressor(build_fn=model_func, batch_size=256, verbose=1)


Epoch 1/50
34/34 [==============================] - 6s 147ms/step - loss: 0.7253 - acc: 0.5132
Epoch 2/50
34/34 [==============================] - 5s 148ms/step - loss: 0.7015 - acc: 0.5166
Epoch 3/50
34/34 [==============================] - 5s 150ms/step - loss: 0.6916 - acc: 0.5354
Epoch 4/50
34/34 [==============================] - 5s 149ms/step - loss: 0.6930 - acc: 0.5388
Epoch 5/50
34/34 [==============================] - 5s 152ms/step - loss: 0.6916 - acc: 0.5450
Epoch 6/50
34/34 [==============================] - 5s 149ms/step - loss: 0.6916 - acc: 0.5417
Epoch 7/50
34/34 [==============================] - 5s 151ms/step - loss: 0.6904 - acc: 0.5466
Epoch 8/50
34/34 [==============================] - 5s 147ms/step - loss: 0.6901 - acc: 0.5447
Epoch 9/50
34/34 [==============================] - 5s 151ms/step - loss: 0.6895 - acc: 0.5439
Epoch 10/50
34/34 [==============================] - 5s 148ms/step - loss: 0.6884 - acc: 0.5483
Epoch 11/50
34/34 [==============================

34/34 [==============================] - 5s 148ms/step - loss: 0.6423 - acc: 0.6182
Epoch 36/50
34/34 [==============================] - 5s 148ms/step - loss: 0.6286 - acc: 0.6303
Epoch 37/50
34/34 [==============================] - 5s 149ms/step - loss: 0.6265 - acc: 0.6420
Epoch 38/50
34/34 [==============================] - 5s 149ms/step - loss: 0.6101 - acc: 0.6516
Epoch 39/50
34/34 [==============================] - 5s 146ms/step - loss: 0.5998 - acc: 0.6716
Epoch 40/50
34/34 [==============================] - 5s 150ms/step - loss: 0.5954 - acc: 0.6831
Epoch 41/50
34/34 [==============================] - 5s 149ms/step - loss: 0.5972 - acc: 0.6773
Epoch 42/50
34/34 [==============================] - 5s 149ms/step - loss: 0.5834 - acc: 0.6832
Epoch 43/50
34/34 [==============================] - 5s 149ms/step - loss: 0.5683 - acc: 0.7074
Epoch 44/50
34/34 [==============================] - 5s 150ms/step - loss: 0.5571 - acc: 0.7142
Epoch 45/50
34/34 [==============================] -

9/9 [==============================] - 1s 44ms/step - loss: 0.9851 - acc: 0.4912
layer_1 4096 layer_2 1024 layer_3 256 layer_4 64 layer_5 16
Epoch 1/50
34/34 [==============================] - 6s 150ms/step - loss: 0.7367 - acc: 0.5159
Epoch 2/50
34/34 [==============================] - 5s 149ms/step - loss: 0.7037 - acc: 0.5284
Epoch 3/50
34/34 [==============================] - 5s 150ms/step - loss: 0.6959 - acc: 0.5354
Epoch 4/50
34/34 [==============================] - 5s 151ms/step - loss: 0.6947 - acc: 0.5417
Epoch 5/50
34/34 [==============================] - 5s 152ms/step - loss: 0.6937 - acc: 0.5407
Epoch 6/50
34/34 [==============================] - 5s 148ms/step - loss: 0.6930 - acc: 0.5363
Epoch 7/50
34/34 [==============================] - 5s 150ms/step - loss: 0.6911 - acc: 0.5411
Epoch 8/50
34/34 [==============================] - 5s 152ms/step - loss: 0.6895 - acc: 0.5460
Epoch 9/50
34/34 [==============================] - 5s 150ms/step - loss: 0.6915 - acc: 0.5452
Epoc

...

C:\Users\Henry\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Henry\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Henry\anaconda3\lib\site-packages\keras\wrappers\scikit_learn.py", line 155, in fit
    self.model = self.build_fn(**self.filter_sk_params(self.build_fn))
  File "<ipython-input-9-beb3aba4a063>", line 8, in model_func
    model.add(Dropout(drop))
  File "C:\Users\Henry\anaconda3\lib\site-packages\keras\layers\core\dropout.py", line 80, in __init__
    raise ValueError(f'Invalid value {rate} received for '
ValueError: Invalid value 85 received for `rate`, expected a value between 0 and 1.

  warnings.warn("Estimator fit failed. The score on this train-test

layer_1 4096 layer_2 1024 layer_3 256 layer_4 64 layer_5 16
layer_1 4096 layer_2 1024 layer_3 256 layer_4 64 layer_5 16


C:\Users\Henry\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Henry\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Henry\anaconda3\lib\site-packages\keras\wrappers\scikit_learn.py", line 155, in fit
    self.model = self.build_fn(**self.filter_sk_params(self.build_fn))
  File "<ipython-input-9-beb3aba4a063>", line 8, in model_func
    model.add(Dropout(drop))
  File "C:\Users\Henry\anaconda3\lib\site-packages\keras\layers\core\dropout.py", line 80, in __init__
    raise ValueError(f'Invalid value {rate} received for '
ValueError: Invalid value 85 received for `rate`, expected a value between 0 and 1.

  warnings.warn("Estimator fit failed. The score on this train-test

Epoch 1/50
42/42 [==============================] - 7s 149ms/step - loss: 0.7375 - acc: 0.5135
Epoch 2/50
42/42 [==============================] - 6s 150ms/step - loss: 0.7017 - acc: 0.4986
Epoch 3/50
42/42 [==============================] - 6s 150ms/step - loss: 0.6957 - acc: 0.5280
Epoch 4/50
42/42 [==============================] - 6s 154ms/step - loss: 0.6935 - acc: 0.5466
Epoch 5/50
42/42 [==============================] - 6s 152ms/step - loss: 0.6929 - acc: 0.5418
Epoch 6/50
42/42 [==============================] - 6s 149ms/step - loss: 0.6936 - acc: 0.5431
Epoch 7/50
42/42 [==============================] - 6s 151ms/step - loss: 0.6915 - acc: 0.5458
Epoch 8/50
42/42 [==============================] - 6s 150ms/step - loss: 0.6895 - acc: 0.5454
Epoch 9/50
42/42 [==============================] - 6s 149ms/step - loss: 0.6897 - acc: 0.5442
Epoch 10/50
42/42 [==============================] - 6s 152ms/step - loss: 0.6898 - acc: 0.5457
Epoch 11/50
42/42 [==============================

{'drop': 0.8,
 'epochs': 50,
 'l1': 4096,
 'l2': 1024,
 'l3': 256,
 'l4': 64,
 'l5': 16}

===

### Model 3

In [11]:
def model_func(l1=32, l2=32, l3=32, l4=32, l5=32, l6=32, l7=32, l8=32, l9=32, l10=32, drop=.5):
    
    print('layer_1', l1, 'layer_2', l2, 'layer_3', l3, 'layer_4', l4, 'layer_5', l5,
         'layer_6', l6, 'layer_7', l7, 'layer_8', l8, 'layer_9', l9, 'layer_10', l10)
    
    model = Sequential()
    
    model.add(Dense(l1, activation='relu', input_shape=(5166,)))
    model.add(Dropout(drop))
    
    model.add(Dense(l2, activation='relu'))
    model.add(Dropout(drop))
 
    model.add(Dense(l3, activation='relu'))
    model.add(Dropout(drop))

    model.add(Dense(l4, activation='relu'))
    model.add(Dropout(drop))

    model.add(Dense(l5, activation='relu'))
    model.add(Dropout(drop))    
    
    model.add(Dense(l6, activation='relu'))
    model.add(Dropout(drop))
 
    model.add(Dense(l7, activation='relu'))
    model.add(Dropout(drop))

    model.add(Dense(l8, activation='relu'))
    model.add(Dropout(drop))

    model.add(Dense(l9, activation='relu'))
    model.add(Dropout(drop)) 

    model.add(Dense(l10, activation='relu'))
    model.add(Dropout(drop)) 
    
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics='acc')
    
    return model

nn = KerasRegressor(build_fn=model_func, batch_size=256, verbose=0)

params = {
    'epochs': [50],
    'l1': [4096],
    'l2': [2048],
    'l3': [1024],
    'l4': [512],
    'l5': [256],
    'l6': [128],
    'l7': [64],
    'l8': [32],
    'l9': [16],
    'l10': [8],
    'drop': [.5, .6, .7]
}

gs3 = GridSearchCV(nn, param_grid=params, cv=5)

gs3.fit(X_train, y_train)

print(gs3.best_score_) # <- negative mean squared error, ie the higher (less negative) the better
gs3.best_params_

layer_1 4096 layer_2 2048 layer_3 1024 layer_4 512 layer_5 256 layer_6 128 layer_7 64 layer_8 32 layer_9 16 layer_10 8


<ipython-input-11-3bbc520ff0b0>:44: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  nn = KerasRegressor(build_fn=model_func, batch_size=256, verbose=1)


Epoch 1/50
34/34 [==============================] - 8s 203ms/step - loss: 0.7019 - acc: 0.5182
Epoch 2/50
34/34 [==============================] - 7s 204ms/step - loss: 0.6917 - acc: 0.5370
Epoch 3/50
34/34 [==============================] - 7s 204ms/step - loss: 0.6893 - acc: 0.5431
Epoch 4/50
34/34 [==============================] - 7s 205ms/step - loss: 0.6897 - acc: 0.5419
Epoch 5/50
34/34 [==============================] - 7s 207ms/step - loss: 0.6875 - acc: 0.5477
Epoch 6/50
34/34 [==============================] - 7s 204ms/step - loss: 0.6875 - acc: 0.5441
Epoch 7/50
34/34 [==============================] - 7s 206ms/step - loss: 0.6821 - acc: 0.5442 1s - loss: 0.6817 - acc: 
Epoch 8/50
34/34 [==============================] - 7s 201ms/step - loss: 0.6796 - acc: 0.5449
Epoch 9/50
34/34 [==============================] - 7s 204ms/step - loss: 0.6677 - acc: 0.5433
Epoch 10/50
34/34 [==============================] - 7s 204ms/step - loss: 0.6542 - acc: 0.5438
Epoch 11/50
34/34 [====

34/34 [==============================] - 7s 199ms/step - loss: 0.5026 - acc: 0.7718
Epoch 14/50
34/34 [==============================] - 7s 199ms/step - loss: 0.4846 - acc: 0.7788
Epoch 15/50
34/34 [==============================] - 7s 199ms/step - loss: 0.4740 - acc: 0.7838
Epoch 16/50
34/34 [==============================] - 7s 205ms/step - loss: 0.4603 - acc: 0.7906
Epoch 17/50
34/34 [==============================] - 7s 201ms/step - loss: 0.4534 - acc: 0.7960
Epoch 18/50
34/34 [==============================] - 7s 203ms/step - loss: 0.4547 - acc: 0.8011
Epoch 19/50
34/34 [==============================] - 7s 201ms/step - loss: 0.4478 - acc: 0.7956
Epoch 20/50
34/34 [==============================] - 7s 202ms/step - loss: 0.4356 - acc: 0.8023
Epoch 21/50
34/34 [==============================] - 7s 199ms/step - loss: 0.4405 - acc: 0.7930
Epoch 22/50
34/34 [==============================] - 7s 198ms/step - loss: 0.4430 - acc: 0.7962
Epoch 23/50
34/34 [==============================] -

34/34 [==============================] - 7s 202ms/step - loss: 0.3845 - acc: 0.8395
Epoch 47/50
34/34 [==============================] - 7s 203ms/step - loss: 0.3722 - acc: 0.8408
Epoch 48/50
34/34 [==============================] - 7s 201ms/step - loss: 0.3708 - acc: 0.8425
Epoch 49/50
34/34 [==============================] - 7s 198ms/step - loss: 0.3647 - acc: 0.8408
Epoch 50/50
9/9 [==============================] - 1s 60ms/step - loss: 2.2982 - acc: 0.4972
layer_1 4096 layer_2 2048 layer_3 1024 layer_4 512 layer_5 256 layer_6 128 layer_7 64 layer_8 32 layer_9 16 layer_10 8
Epoch 1/50
34/34 [==============================] - 8s 199ms/step - loss: 0.7146 - acc: 0.5118
Epoch 2/50
34/34 [==============================] - 7s 205ms/step - loss: 0.6941 - acc: 0.5322
Epoch 3/50
34/34 [==============================] - 7s 207ms/step - loss: 0.6905 - acc: 0.5488
Epoch 4/50
34/34 [==============================] - 7s 202ms/step - loss: 0.6913 - acc: 0.5434
Epoch 5/50
34/34 [==================

34/34 [==============================] - 7s 200ms/step - loss: 0.6900 - acc: 0.5440
Epoch 6/50
34/34 [==============================] - 7s 204ms/step - loss: 0.6904 - acc: 0.5449
Epoch 7/50
34/34 [==============================] - 7s 202ms/step - loss: 0.6896 - acc: 0.5437
Epoch 8/50
34/34 [==============================] - 7s 202ms/step - loss: 0.6900 - acc: 0.5435
Epoch 9/50
34/34 [==============================] - 7s 206ms/step - loss: 0.6896 - acc: 0.5440
Epoch 10/50
34/34 [==============================] - 7s 203ms/step - loss: 0.6888 - acc: 0.5443
Epoch 11/50
34/34 [==============================] - 7s 200ms/step - loss: 0.6886 - acc: 0.5438
Epoch 12/50
34/34 [==============================] - 7s 200ms/step - loss: 0.6882 - acc: 0.5435
Epoch 13/50
34/34 [==============================] - 7s 201ms/step - loss: 0.6872 - acc: 0.5429 3s - loss: 0.6877 - a - ETA: 1s - loss: 0.6865 - acc:
Epoch 14/50
34/34 [==============================] - 7s 200ms/step - loss: 0.6858 - acc: 0.5436 1s

34/34 [==============================] - 7s 203ms/step - loss: 0.5005 - acc: 0.7461
Epoch 38/50
34/34 [==============================] - 7s 200ms/step - loss: 0.5006 - acc: 0.7481
Epoch 39/50
34/34 [==============================] - 7s 201ms/step - loss: 0.5030 - acc: 0.7479
Epoch 40/50
34/34 [==============================] - 7s 203ms/step - loss: 0.4896 - acc: 0.7571
Epoch 41/50
34/34 [==============================] - 7s 205ms/step - loss: 0.4796 - acc: 0.7641
Epoch 42/50
34/34 [==============================] - 7s 202ms/step - loss: 0.4786 - acc: 0.7667
Epoch 43/50
34/34 [==============================] - 7s 204ms/step - loss: 0.4715 - acc: 0.7723 4s - loss
Epoch 44/50
34/34 [==============================] - 7s 201ms/step - loss: 0.4682 - acc: 0.7753
Epoch 45/50
34/34 [==============================] - 7s 199ms/step - loss: 0.4601 - acc: 0.7774
Epoch 46/50
34/34 [==============================] - 7s 206ms/step - loss: 0.4596 - acc: 0.7823
Epoch 47/50
34/34 [=======================

9/9 [==============================] - 1s 55ms/step - loss: 0.6905 - acc: 0.5382
layer_1 4096 layer_2 2048 layer_3 1024 layer_4 512 layer_5 256 layer_6 128 layer_7 64 layer_8 32 layer_9 16 layer_10 8
Epoch 1/50
34/34 [==============================] - 8s 201ms/step - loss: 0.8124 - acc: 0.5112
Epoch 2/50
34/34 [==============================] - 7s 204ms/step - loss: 0.7063 - acc: 0.5244
Epoch 3/50
34/34 [==============================] - 7s 198ms/step - loss: 0.6971 - acc: 0.5301
Epoch 4/50
34/34 [==============================] - 7s 201ms/step - loss: 0.6930 - acc: 0.5433
Epoch 5/50
34/34 [==============================] - 7s 202ms/step - loss: 0.6923 - acc: 0.5375
Epoch 6/50
34/34 [==============================] - 7s 199ms/step - loss: 0.6923 - acc: 0.5396
Epoch 7/50
34/34 [==============================] - 7s 199ms/step - loss: 0.6913 - acc: 0.5429
Epoch 8/50
34/34 [==============================] - 7s 201ms/step - loss: 0.6909 - acc: 0.5440
Epoch 9/50
34/34 [======================

{'drop': 0.7,
 'epochs': 50,
 'l1': 4096,
 'l10': 8,
 'l2': 2048,
 'l3': 1024,
 'l4': 512,
 'l5': 256,
 'l6': 128,
 'l7': 64,
 'l8': 32,
 'l9': 16}

===

### Model 4

In [13]:
def model_func(l1=32, l2=32, l3=32, l4=32, l5=32, drop=.5):
    
    print('layer_1', l1, 'layer_2', l2, 'layer_3', l3, 'layer_4', l4, 'layer_5', l5)
    
    model = Sequential()
    
    model.add(Dense(l1, activation='relu', input_shape=(5166,)))
    model.add(Dropout(drop))
    
    model.add(Dense(l2, activation='relu'))
    model.add(Dropout(drop))
 
    model.add(Dense(l3, activation='relu'))
    model.add(Dropout(drop))

    model.add(Dense(l4, activation='relu'))
    model.add(Dropout(drop))

    model.add(Dense(l5, activation='relu'))
    model.add(Dropout(drop))    

    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics='acc')
    
    return model

nn = KerasRegressor(build_fn=model_func, batch_size=256, verbose=0)

params = {
    'epochs': [50],
    'l1': [4096],
    'l2': [1024],
    'l3': [256],
    'l4': [64],
    'l5': [16],
    'drop': [.65, .7, .75, .8, .85]
}

gs4 = GridSearchCV(nn, param_grid=params, cv=5)

gs4.fit(X_train, y_train)

print(gs4.best_score_) # <- negative mean squared error, ie the higher (less negative) the better
gs4.best_params_

layer_1 4096 layer_2 1024 layer_3 256 layer_4 64 layer_5 16


<ipython-input-13-c73e0408d965>:28: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  nn = KerasRegressor(build_fn=model_func, batch_size=256, verbose=1)


Epoch 1/50
34/34 [==============================] - 6s 146ms/step - loss: 0.7002 - acc: 0.5163
Epoch 2/50
34/34 [==============================] - 5s 149ms/step - loss: 0.6929 - acc: 0.5258
Epoch 3/50
34/34 [==============================] - 5s 149ms/step - loss: 0.6907 - acc: 0.5336
Epoch 4/50
34/34 [==============================] - 5s 149ms/step - loss: 0.6908 - acc: 0.5339
Epoch 5/50
34/34 [==============================] - 5s 150ms/step - loss: 0.6882 - acc: 0.5393
Epoch 6/50
34/34 [==============================] - 5s 152ms/step - loss: 0.6879 - acc: 0.5405
Epoch 7/50
34/34 [==============================] - 5s 150ms/step - loss: 0.6847 - acc: 0.5429
Epoch 8/50
34/34 [==============================] - 5s 152ms/step - loss: 0.6822 - acc: 0.5477
Epoch 9/50
34/34 [==============================] - 5s 150ms/step - loss: 0.6805 - acc: 0.5456
Epoch 10/50
34/34 [==============================] - 5s 150ms/step - loss: 0.6761 - acc: 0.5454
Epoch 11/50
34/34 [==============================

34/34 [==============================] - 5s 149ms/step - loss: 0.5035 - acc: 0.7775
Epoch 17/50
34/34 [==============================] - 5s 150ms/step - loss: 0.4597 - acc: 0.8022
Epoch 18/50
34/34 [==============================] - 5s 147ms/step - loss: 0.4359 - acc: 0.8105
Epoch 19/50
34/34 [==============================] - 5s 147ms/step - loss: 0.3945 - acc: 0.8196
Epoch 20/50
34/34 [==============================] - 5s 150ms/step - loss: 0.3807 - acc: 0.8254
Epoch 21/50
34/34 [==============================] - 5s 149ms/step - loss: 0.3573 - acc: 0.8301
Epoch 22/50
34/34 [==============================] - 5s 150ms/step - loss: 0.3547 - acc: 0.8360
Epoch 23/50
34/34 [==============================] - 5s 149ms/step - loss: 0.3408 - acc: 0.8375
Epoch 24/50
34/34 [==============================] - 5s 148ms/step - loss: 0.3241 - acc: 0.8396
Epoch 25/50
34/34 [==============================] - 5s 150ms/step - loss: 0.3126 - acc: 0.8485
Epoch 26/50
34/34 [==============================] -

9/9 [==============================] - 1s 40ms/step - loss: 3.5142 - acc: 0.5568
layer_1 4096 layer_2 1024 layer_3 256 layer_4 64 layer_5 16
Epoch 1/50
34/34 [==============================] - 6s 150ms/step - loss: 0.7073 - acc: 0.5121
Epoch 2/50
34/34 [==============================] - 5s 157ms/step - loss: 0.6957 - acc: 0.5199
Epoch 3/50
34/34 [==============================] - 5s 151ms/step - loss: 0.6930 - acc: 0.5340
Epoch 4/50
34/34 [==============================] - 5s 147ms/step - loss: 0.6912 - acc: 0.5391
Epoch 5/50
34/34 [==============================] - 5s 154ms/step - loss: 0.6906 - acc: 0.5420
Epoch 6/50
34/34 [==============================] - 5s 153ms/step - loss: 0.6900 - acc: 0.5411
Epoch 7/50
34/34 [==============================] - 5s 152ms/step - loss: 0.6903 - acc: 0.5419
Epoch 8/50
34/34 [==============================] - 5s 150ms/step - loss: 0.6894 - acc: 0.5442
Epoch 9/50
34/34 [==============================] - 5s 151ms/step - loss: 0.6900 - acc: 0.5445
Epoc

...

34/34 [==============================] - 5s 148ms/step - loss: 0.5238 - acc: 0.7316
Epoch 47/50
34/34 [==============================] - 5s 146ms/step - loss: 0.5239 - acc: 0.7373
Epoch 48/50
34/34 [==============================] - 5s 145ms/step - loss: 0.5138 - acc: 0.7446
Epoch 49/50
34/34 [==============================] - 5s 146ms/step - loss: 0.4973 - acc: 0.7490
Epoch 50/50
9/9 [==============================] - 1s 37ms/step - loss: 1.0039 - acc: 0.5084
layer_1 4096 layer_2 1024 layer_3 256 layer_4 64 layer_5 16
Epoch 1/50
34/34 [==============================] - 6s 144ms/step - loss: 0.7361 - acc: 0.5126
Epoch 2/50
34/34 [==============================] - 5s 148ms/step - loss: 0.7066 - acc: 0.5208
Epoch 3/50
34/34 [==============================] - 5s 147ms/step - loss: 0.6966 - acc: 0.5329
Epoch 4/50
34/34 [==============================] - 5s 147ms/step - loss: 0.6944 - acc: 0.5335
Epoch 5/50
34/34 [==============================] - 5s 144ms/step - loss: 0.6923 - acc: 0.5385


34/34 [==============================] - 5s 147ms/step - loss: 0.6910 - acc: 0.5407
Epoch 11/50
34/34 [==============================] - 5s 145ms/step - loss: 0.6904 - acc: 0.5386
Epoch 12/50
34/34 [==============================] - 5s 149ms/step - loss: 0.6899 - acc: 0.5414
Epoch 13/50
34/34 [==============================] - 5s 146ms/step - loss: 0.6923 - acc: 0.5392
Epoch 14/50
34/34 [==============================] - 5s 146ms/step - loss: 0.6914 - acc: 0.5400
Epoch 15/50
34/34 [==============================] - 5s 148ms/step - loss: 0.6900 - acc: 0.5437
Epoch 16/50
34/34 [==============================] - 5s 148ms/step - loss: 0.6896 - acc: 0.5438
Epoch 17/50
34/34 [==============================] - 5s 149ms/step - loss: 0.6899 - acc: 0.5402
Epoch 18/50
34/34 [==============================] - 5s 146ms/step - loss: 0.6897 - acc: 0.5425
Epoch 19/50
34/34 [==============================] - 5s 149ms/step - loss: 0.6906 - acc: 0.5414
Epoch 20/50
34/34 [==============================] -

34/34 [==============================] - 5s 148ms/step - loss: 0.6806 - acc: 0.5456
Epoch 45/50
34/34 [==============================] - 5s 144ms/step - loss: 0.6813 - acc: 0.5459
Epoch 46/50
34/34 [==============================] - 5s 147ms/step - loss: 0.6785 - acc: 0.5458
Epoch 47/50
34/34 [==============================] - 5s 147ms/step - loss: 0.6801 - acc: 0.5452
Epoch 48/50
34/34 [==============================] - 5s 147ms/step - loss: 0.6731 - acc: 0.5468
Epoch 49/50
34/34 [==============================] - 5s 145ms/step - loss: 0.6708 - acc: 0.5460
Epoch 50/50
9/9 [==============================] - 1s 42ms/step - loss: 0.6827 - acc: 0.5517
layer_1 4096 layer_2 1024 layer_3 256 layer_4 64 layer_5 16
Epoch 1/50
34/34 [==============================] - 5s 145ms/step - loss: 0.8546 - acc: 0.4953
Epoch 2/50
34/34 [==============================] - 5s 147ms/step - loss: 0.7319 - acc: 0.5190
Epoch 3/50
34/34 [==============================] - 5s 145ms/step - loss: 0.7130 - acc: 0.532

34/34 [==============================] - 5s 146ms/step - loss: 0.6922 - acc: 0.5402
Epoch 9/50
34/34 [==============================] - 5s 147ms/step - loss: 0.6931 - acc: 0.5366
Epoch 10/50
34/34 [==============================] - 5s 147ms/step - loss: 0.6914 - acc: 0.5452
Epoch 11/50
34/34 [==============================] - 5s 147ms/step - loss: 0.6935 - acc: 0.5364
Epoch 12/50
34/34 [==============================] - 5s 148ms/step - loss: 0.6937 - acc: 0.5365
Epoch 13/50
34/34 [==============================] - 5s 147ms/step - loss: 0.6907 - acc: 0.5416
Epoch 14/50
34/34 [==============================] - 5s 147ms/step - loss: 0.6911 - acc: 0.5407
Epoch 15/50
34/34 [==============================] - 5s 147ms/step - loss: 0.6909 - acc: 0.5433
Epoch 16/50
34/34 [==============================] - 5s 147ms/step - loss: 0.6911 - acc: 0.5422
Epoch 17/50
34/34 [==============================] - 5s 146ms/step - loss: 0.6907 - acc: 0.5451
Epoch 18/50
34/34 [==============================] - 

{'drop': 0.85,
 'epochs': 50,
 'l1': 4096,
 'l2': 1024,
 'l3': 256,
 'l4': 64,
 'l5': 16}

### Model 5

In [8]:
def model_func(l1=32, l2=32, l3=32, l4=32, l5=32, drop=.5):
    
    print('layer_1', l1, 'layer_2', l2, 'layer_3', l3, 'layer_4', l4, 'layer_5', l5)
    
    model = Sequential()
    
    model.add(Dense(l1, activation='relu', input_shape=(5166,)))
    model.add(Dropout(drop))
    
    model.add(Dense(l2, activation='relu'))
    model.add(Dropout(drop))
 
    model.add(Dense(l3, activation='relu'))
    model.add(Dropout(drop))

    model.add(Dense(l4, activation='relu'))
    model.add(Dropout(drop))

    model.add(Dense(l5, activation='relu'))
    model.add(Dropout(drop))    

    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics='acc')
    
    return model

nn = KerasRegressor(build_fn=model_func, batch_size=256, verbose=0)

params = {
    'epochs': [20],
    'l1': [4096, 2048, 1024],
    'l2': [2048, 1024, 512],
    'l3': [1024, 512, 256],
    'drop': [.7, .75, .8, .85]
}

gs5 = GridSearchCV(nn, param_grid=params, cv=5)

gs5.fit(X_train, y_train)

print(gs.best_score_) # <- negative mean squared error, ie the higher (less negative) the better
gs5.best_params_

<ipython-input-8-a319c50bd7da>:28: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  nn = KerasRegressor(build_fn=model_func, batch_size=256, verbose=1)


layer_1 4096 layer_2 2048 layer_3 1024 layer_4 32 layer_5 32
Epoch 1/20
34/34 [==============================] - 7s 186ms/step - loss: 0.7015 - acc: 0.5156
Epoch 2/20
34/34 [==============================] - 6s 186ms/step - loss: 0.6923 - acc: 0.5438
Epoch 3/20
34/34 [==============================] - 6s 187ms/step - loss: 0.6897 - acc: 0.5489
Epoch 4/20
34/34 [==============================] - 6s 191ms/step - loss: 0.6894 - acc: 0.5486
Epoch 5/20
34/34 [==============================] - 6s 185ms/step - loss: 0.6896 - acc: 0.5518 5s
Epoch 6/20
34/34 [==============================] - 6s 185ms/step - loss: 0.6897 - acc: 0.5482
Epoch 7/20
34/34 [==============================] - 6s 185ms/step - loss: 0.6889 - acc: 0.5477
Epoch 8/20
34/34 [==============================] - 6s 186ms/step - loss: 0.6894 - acc: 0.5492
Epoch 9/20
34/34 [==============================] - 6s 186ms/step - loss: 0.6886 - acc: 0.5488
Epoch 10/20
34/34 [==============================] - 6s 186ms/step - loss: 0.6882

34/34 [==============================] - 6s 188ms/step - loss: 0.6480 - acc: 0.5482
Epoch 20/20
9/9 [==============================] - 1s 48ms/step - loss: 0.7021 - acc: 0.5382
layer_1 4096 layer_2 2048 layer_3 512 layer_4 32 layer_5 32
Epoch 1/20
34/34 [==============================] - 7s 183ms/step - loss: 0.7037 - acc: 0.5180 2s - loss: 0.7065
Epoch 2/20
34/34 [==============================] - 6s 186ms/step - loss: 0.6937 - acc: 0.5367
Epoch 3/20
34/34 [==============================] - 6s 183ms/step - loss: 0.6945 - acc: 0.5387
Epoch 4/20
34/34 [==============================] - 6s 187ms/step - loss: 0.6907 - acc: 0.5406
Epoch 5/20
34/34 [==============================] - 6s 186ms/step - loss: 0.6907 - acc: 0.5424
Epoch 6/20
34/34 [==============================] - 7s 195ms/step - loss: 0.6895 - acc: 0.5479 
Epoch 7/20
34/34 [==============================] - 7s 190ms/step - loss: 0.6905 - acc: 0.5428
Epoch 8/20
34/34 [==============================] - 6s 184ms/step - loss: 0.689

34/34 [==============================] - 6s 163ms/step - loss: 0.6829 - acc: 0.5481
Epoch 18/20
34/34 [==============================] - 6s 164ms/step - loss: 0.6779 - acc: 0.5462
Epoch 19/20
34/34 [==============================] - 6s 176ms/step - loss: 0.6754 - acc: 0.5478
Epoch 20/20
9/9 [==============================] - 1s 44ms/step - loss: 0.6816 - acc: 0.5407
layer_1 4096 layer_2 1024 layer_3 1024 layer_4 32 layer_5 32
Epoch 1/20
34/34 [==============================] - 7s 170ms/step - loss: 0.6991 - acc: 0.5144
Epoch 2/20
34/34 [==============================] - 6s 164ms/step - loss: 0.6938 - acc: 0.5381
Epoch 3/20
34/34 [==============================] - 6s 166ms/step - loss: 0.6913 - acc: 0.5416
Epoch 4/20
34/34 [==============================] - 6s 165ms/step - loss: 0.6901 - acc: 0.5418
Epoch 5/20
34/34 [==============================] - 6s 162ms/step - loss: 0.6902 - acc: 0.5446
Epoch 6/20
34/34 [==============================] - 6s 163ms/step - loss: 0.6891 - acc: 0.5460


34/34 [==============================] - 6s 165ms/step - loss: 0.6818 - acc: 0.5422 2s - loss: 0.
Epoch 15/20
34/34 [==============================] - 6s 167ms/step - loss: 0.6762 - acc: 0.5416
Epoch 16/20
34/34 [==============================] - 6s 172ms/step - loss: 0.6781 - acc: 0.5431
Epoch 17/20
34/34 [==============================] - 6s 169ms/step - loss: 0.6705 - acc: 0.5416
Epoch 18/20
34/34 [==============================] - 6s 165ms/step - loss: 0.6649 - acc: 0.5413
Epoch 19/20
34/34 [==============================] - 6s 164ms/step - loss: 0.6519 - acc: 0.5442
Epoch 20/20
9/9 [==============================] - 1s 42ms/step - loss: 0.6981 - acc: 0.5577
layer_1 4096 layer_2 1024 layer_3 512 layer_4 32 layer_5 32
Epoch 1/20
34/34 [==============================] - 6s 162ms/step - loss: 0.7018 - acc: 0.5147
Epoch 2/20
34/34 [==============================] - 6s 165ms/step - loss: 0.6930 - acc: 0.5326
Epoch 3/20
34/34 [==============================] - 6s 165ms/step - loss: 0.692

34/34 [==============================] - 5s 150ms/step - loss: 0.6885 - acc: 0.5451
Epoch 13/20
34/34 [==============================] - 5s 158ms/step - loss: 0.6860 - acc: 0.5461
Epoch 14/20
34/34 [==============================] - 6s 166ms/step - loss: 0.6835 - acc: 0.5447
Epoch 15/20
34/34 [==============================] - 5s 152ms/step - loss: 0.6826 - acc: 0.5451
Epoch 16/20
34/34 [==============================] - 5s 157ms/step - loss: 0.6781 - acc: 0.5445
Epoch 17/20
34/34 [==============================] - 5s 153ms/step - loss: 0.6729 - acc: 0.5449
Epoch 18/20
34/34 [==============================] - 5s 154ms/step - loss: 0.6668 - acc: 0.5460 1s - loss: 0.6659 - a
Epoch 19/20
34/34 [==============================] - 5s 154ms/step - loss: 0.6624 - acc: 0.5466
Epoch 20/20
9/9 [==============================] - 1s 47ms/step - loss: 0.6927 - acc: 0.5507
layer_1 4096 layer_2 512 layer_3 1024 layer_4 32 layer_5 32
Epoch 1/20
34/34 [==============================] - 6s 158ms/step - l

KeyboardInterrupt: 

### 1 Feature Model

In [6]:
# getting dummie variables
df_dummies = df.filter(['flight_condition'], axis=1)
df_dummies = pd.get_dummies(df_dummies, columns=['flight_condition'], drop_first=True)

In [7]:
X = df_dummies.values
y = df['deaths_and_injuries'].astype(int).values

In [8]:
# train test split and scale
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [9]:
X_train[0].shape[0]

2

In [10]:
def model_func(l1=32, l2=32, l3=32, l4=32, l5=32, drop=.5):
    
    print('layer_1', l1, 'layer_2', l2, 'layer_3', l3, 'layer_4', l4, 'layer_5', l5)
    model = Sequential()
    model.add(Dense(l1, activation='relu', input_shape=(2,)))
    model.add(Dropout(drop))
    
    model.add(Dense(l2, activation='relu'))
    model.add(Dropout(drop))
 
    model.add(Dense(l3, activation='relu'))
    model.add(Dropout(drop))

    model.add(Dense(l4, activation='relu'))
    model.add(Dropout(drop))

    model.add(Dense(l5, activation='relu'))
    model.add(Dropout(drop))    

    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics='acc')
    
    return model

nn = KerasRegressor(build_fn=model_func, batch_size=32, verbose=0)

params = {
    'epochs': [20],
    'l1': [32, 16, 8],
    'l2': [32, 16, 8],
    'l3': [32, 16, 8],
    'drop': [.5, 6, .7, .75, .8, .85]
}

gs5 = GridSearchCV(nn, param_grid=params, cv=5)

gs5.fit(X_train, y_train)

print(gs.best_score_) 
gs5.best_params_

<ipython-input-10-62de10bec50a>:26: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  nn = KerasRegressor(build_fn=model_func, batch_size=32, verbose=1)


layer_1 32 layer_2 32 layer_3 32 layer_4 32 layer_5 32
Epoch 1/20
269/269 [==============================] - 1s 1ms/step - loss: 0.6953 - acc: 0.5203
Epoch 2/20
269/269 [==============================] - 0s 1ms/step - loss: 0.6884 - acc: 0.5482
Epoch 3/20
269/269 [==============================] - 0s 1ms/step - loss: 0.6883 - acc: 0.5482
Epoch 4/20
269/269 [==============================] - 0s 989us/step - loss: 0.6893 - acc: 0.5503
Epoch 5/20
269/269 [==============================] - 0s 1ms/step - loss: 0.6882 - acc: 0.5492
Epoch 6/20
269/269 [==============================] - 0s 1ms/step - loss: 0.6886 - acc: 0.5490
Epoch 7/20
269/269 [==============================] - 0s 1ms/step - loss: 0.6884 - acc: 0.5493
Epoch 8/20
269/269 [==============================] - 0s 1ms/step - loss: 0.6873 - acc: 0.5497
Epoch 9/20
269/269 [==============================] - 0s 1ms/step - loss: 0.6885 - acc: 0.5492
Epoch 10/20
269/269 [==============================] - 0s 1ms/step - loss: 0.6884 - acc:

68/68 [==============================] - 0s 731us/step - loss: 0.6871 - acc: 0.5577
layer_1 32 layer_2 32 layer_3 32 layer_4 32 layer_5 32
Epoch 1/20
269/269 [==============================] - 1s 1ms/step - loss: 0.6926 - acc: 0.5274
Epoch 2/20
269/269 [==============================] - 0s 1ms/step - loss: 0.6899 - acc: 0.5436
Epoch 3/20
269/269 [==============================] - 0s 1ms/step - loss: 0.6882 - acc: 0.5471
Epoch 4/20
269/269 [==============================] - 0s 1ms/step - loss: 0.6884 - acc: 0.5471
Epoch 5/20
269/269 [==============================] - 0s 1ms/step - loss: 0.6893 - acc: 0.5464
Epoch 6/20
269/269 [==============================] - 0s 1ms/step - loss: 0.6881 - acc: 0.5470
Epoch 7/20
269/269 [==============================] - 0s 1ms/step - loss: 0.6878 - acc: 0.5467
Epoch 8/20
269/269 [==============================] - 0s 1ms/step - loss: 0.6886 - acc: 0.5465
Epoch 9/20
269/269 [==============================] - 0s 1ms/step - loss: 0.6882 - acc: 0.5467
Epoch 

269/269 [==============================] - 0s 1ms/step - loss: 0.6880 - acc: 0.5503
Epoch 20/20
68/68 [==============================] - 0s 642us/step - loss: 0.6893 - acc: 0.5382
layer_1 32 layer_2 32 layer_3 16 layer_4 32 layer_5 32
Epoch 1/20
269/269 [==============================] - 1s 1ms/step - loss: 0.6898 - acc: 0.5446
Epoch 2/20
269/269 [==============================] - 0s 1ms/step - loss: 0.6888 - acc: 0.5451
Epoch 3/20
269/269 [==============================] - 0s 1ms/step - loss: 0.6889 - acc: 0.5450
Epoch 4/20
269/269 [==============================] - 0s 974us/step - loss: 0.6887 - acc: 0.5453
Epoch 5/20
269/269 [==============================] - 0s 1ms/step - loss: 0.6893 - acc: 0.5457
Epoch 6/20
269/269 [==============================] - 0s 1ms/step - loss: 0.6888 - acc: 0.5453
Epoch 7/20
269/269 [==============================] - 0s 1ms/step - loss: 0.6883 - acc: 0.5453
Epoch 8/20
269/269 [==============================] - 0s 1ms/step - loss: 0.6890 - acc: 0.5453
Epo

269/269 [==============================] - 0s 1ms/step - loss: 0.6887 - acc: 0.5471
Epoch 19/20
269/269 [==============================] - 0s 1ms/step - loss: 0.6890 - acc: 0.5471
Epoch 20/20
68/68 [==============================] - 0s 657us/step - loss: 0.6876 - acc: 0.5507
layer_1 32 layer_2 32 layer_3 8 layer_4 32 layer_5 32
Epoch 1/20
269/269 [==============================] - 1s 996us/step - loss: 0.6911 - acc: 0.5321
Epoch 2/20
269/269 [==============================] - 0s 1ms/step - loss: 0.6889 - acc: 0.5488
Epoch 3/20
269/269 [==============================] - 0s 1ms/step - loss: 0.6885 - acc: 0.5502
Epoch 4/20
269/269 [==============================] - 0s 1ms/step - loss: 0.6885 - acc: 0.5502
Epoch 5/20
269/269 [==============================] - 0s 1ms/step - loss: 0.6881 - acc: 0.5497
Epoch 6/20
269/269 [==============================] - 0s 974us/step - loss: 0.6885 - acc: 0.5500
Epoch 7/20
269/269 [==============================] - 0s 933us/step - loss: 0.6881 - acc: 0.5504

...

269/269 [==============================] - 0s 884us/step - loss: 0.6898 - acc: 0.5470
Epoch 14/20
269/269 [==============================] - 0s 933us/step - loss: 0.6896 - acc: 0.5471
Epoch 15/20
269/269 [==============================] - 0s 918us/step - loss: 0.6891 - acc: 0.5468 0s - loss: 0.6901 - acc: 0.5
Epoch 16/20
269/269 [==============================] - 0s 884us/step - loss: 0.6888 - acc: 0.5467
Epoch 17/20
269/269 [==============================] - 0s 1ms/step - loss: 0.6890 - acc: 0.5471A: 0s - loss: 0.6881 - acc: 0
Epoch 18/20
269/269 [==============================] - 0s 1ms/step - loss: 0.6890 - acc: 0.5468
Epoch 19/20
269/269 [==============================] - 0s 978us/step - loss: 0.6899 - acc: 0.5470
Epoch 20/20
68/68 [==============================] - 0s 746us/step - loss: 0.6878 - acc: 0.5517
layer_1 8 layer_2 8 layer_3 8 layer_4 32 layer_5 32
Epoch 1/20
269/269 [==============================] - 1s 1ms/step - loss: 0.9749 - acc: 0.5435
Epoch 2/20
269/269 [=========

269/269 [==============================] - 0s 892us/step - loss: 0.6893 - acc: 0.5454
Epoch 10/20
269/269 [==============================] - 0s 903us/step - loss: 0.6890 - acc: 0.5456
Epoch 11/20
269/269 [==============================] - 0s 914us/step - loss: 0.6900 - acc: 0.5453
Epoch 12/20
269/269 [==============================] - 0s 1ms/step - loss: 0.6898 - acc: 0.5452
Epoch 13/20
269/269 [==============================] - 0s 1ms/step - loss: 0.6891 - acc: 0.5458
Epoch 14/20
269/269 [==============================] - 0s 1ms/step - loss: 0.6890 - acc: 0.5454
Epoch 15/20
269/269 [==============================] - 0s 966us/step - loss: 0.6897 - acc: 0.5452
Epoch 16/20
269/269 [==============================] - 0s 907us/step - loss: 0.6892 - acc: 0.5454
Epoch 17/20
269/269 [==============================] - 0s 884us/step - loss: 0.6891 - acc: 0.5453
Epoch 18/20
269/269 [==============================] - 0s 955us/step - loss: 0.6906 - acc: 0.5454
Epoch 19/20
269/269 [=================

C:\Users\Henry\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [-0.68842515 -0.68748543 -0.68716686 -0.68824914 -0.68845705 -0.68846903
 -0.68617833 -0.68733857 -0.6879283  -0.68769196 -0.68814007 -0.68868787
 -0.6884465  -0.68885182 -0.688653   -0.68847879 -0.68694983 -0.68860341
 -0.6868857  -0.68857284 -0.68845379 -0.68856891 -0.68866682 -0.687567
 -0.68835876 -0.68862158 -0.68827267         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
 -0.68867362 -0.68863422 -0.68861701 -0.6886656  -0.68858627 -0.68860742
 -0.68864381 -0.68860759 -0.68860469 -0.68858184 -0.68861647 -0.68860587
 -0.68863177 -0.6886373  -0.68860384 -0.6886632  -0.6885

336/336 [==============================] - 1s 985us/step - loss: 0.6922 - acc: 0.5348
Epoch 2/20
336/336 [==============================] - 0s 1ms/step - loss: 0.6893 - acc: 0.5459
Epoch 3/20
336/336 [==============================] - 0s 1ms/step - loss: 0.6889 - acc: 0.5481
Epoch 4/20
336/336 [==============================] - 0s 955us/step - loss: 0.6886 - acc: 0.5475
Epoch 5/20
336/336 [==============================] - 0s 955us/step - loss: 0.6888 - acc: 0.5472
Epoch 6/20
336/336 [==============================] - 0s 988us/step - loss: 0.6887 - acc: 0.5481
Epoch 7/20
336/336 [==============================] - 0s 1ms/step - loss: 0.6888 - acc: 0.5475
Epoch 8/20
336/336 [==============================] - 0s 1ms/step - loss: 0.6889 - acc: 0.5477
Epoch 9/20
336/336 [==============================] - 0s 919us/step - loss: 0.6887 - acc: 0.5478
Epoch 10/20
336/336 [==============================] - 0s 916us/step - loss: 0.6886 - acc: 0.5478
Epoch 11/20
336/336 [===========================

NameError: name 'gs' is not defined